## Import and load keys

In [ ]:
import os
import requests
from dotenv import load_dotenv
from mistralai import Mistral
import json
import gradio as gr
import random
import time
import traceback

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API de Mistral desde las variables de entorno
api_key = os.getenv('MISTRAL_API_KEY')
client = Mistral(api_key=api_key)
model = "mistral-small-latest"

## World Creation

In [2]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ],
    temperature=0.9
)

In [3]:
world_output =output.choices[0].message.content
print(world_output)
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

World Name: Beastrum

World Description: In the realm of Beastrum, cities are not built on land, but on the backs of colossal, sentient beasts called Terrastrids. These gentle giants roam the ever-shifting landscapes, their movements shaping the world's geography. Humans and other races live in harmony with the Terrastrids, constructing their homes and societies on the beasts' massive shells, forming a unique, mobile civilization. The sky is filled with the silhouettes of these living cities, each one a testament to the symbiotic relationship between its inhabitants and their colossal hosts.


## Kingdom Creation

In [ ]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
    temperature=0.9
)

In [5]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

Created kingdom "Aetheria" in Beastrum
Created kingdom "Verdantia" in Beastrum
Created kingdom "Ignisoria" in Beastrum

Kingdom 1 Description: Ignisoria is a fiery kingdom atop the shell of the powerful and hot-blooded Terrastrid, Pyroclast. King Ember, a formidable warrior and blacksmith, leads his people with strength and passion. Ignisorians are master craftsmen, forging weapons and armor of unparalleled quality. Their history is one of conquest and redemption, as they once sought to dominate other kingdoms but now strive for peace and cooperation. Ignisorians value honor and craftsmanship, with their culture revolving around grand forges and fierce competitions.


## Town Creation

In [6]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom and world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""
    
import time

import time  # Asegúrate de importar la biblioteca time

def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.complete(
      model=model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
      temperature=0.9,
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\nCreating towns for kingdom: {kingdom["name"]}...'):
        for num in range(1, 4):
            print(output.split(f"Town {num} Name: ")[1].split("\n")[0])
            print(output.split(f"Town {num} Description: ")[1].split("\n")[0])
            print("\n")
         
            town_name = output.split(f"Town {num} Name: ")[1].split("\n")[0] 
            town_description = output.split(f"Town {num} Description: ")[1].split("\n")[0]
        
            town = {
              "name": town_name,
              "description": town_description,
              "world": world['name'],
              "kingdom": kingdom['name']
            }
            towns[town_name] = town
        kingdom["towns"] = towns

In [7]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

first_kingdom = list(kingdoms.values())[0]
print(first_kingdom)
town = list(first_kingdom['towns'].values())[0]
print(f'\nTown 1 Description: {town["description"]}')


Creating towns for kingdom: Aetheria...
Lumina
Nestled in the lush, floating islands of the Verdant Expanse, Lumina is a town of light and growth. It's built on the shell of the Terrastrid, Veridia, who loves to bask in the sunlight. The town's important places include the Sun Temple, a grand structure that harnesses solar energy, and the Whispering Gardens, where plants from across Beastrum are cultivated. Lumina's history is filled with tales of the Sun Cult, a group that once tried to control Veridia's movements to keep Lumina in eternal daylight, but were eventually persuaded to live in harmony with their Terrastrid.


Glacialis
Glacialis is a frost-covered town in the icy peaks of the Frostfall Mountains, atop the Terrastrid, Cryon. The town is known for its ice sculptures and the Frozen Library, which holds ancient texts preserved in ice. The town's interesting history revolves around the Ice Harvesters, a group that once mined the Terrastrid's frozen tears for their magical pro

## Npcs Creation

In [8]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""
    

In [9]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.complete(
      model= model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
      ],
        temperature=0.7 
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for num in range(1, 4):
        print(npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0])
        print(npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0])
        print("\n")

        character_name = npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0]
        character_description = npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0]

        character = {
        "name": character_name,
        "description": character_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[character_name] = character
        town["npcs"] = npcs

    
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom and town
    break


Creating characters for the town of: Lumina...
Eamon Sunweaver
Eamon is a lanky, sun-kissed Lumina native with bright green eyes and hair adorned with small, glowing plants. He's a skilled Solar Engineer, designing devices that harness and manipulate sunlight. Deep down, Eamon struggles with the legacy of the Sun Cult, his ancestors were part of the group that tried to control Veridia. He desires to redeem their name by using his inventions to benefit Lumina and all of Beastrum, fostering harmony with the Terrastrids.


Lyra Whisperwind
Lyra is a petite, agile woman with silver hair and eyes that seem to hold the wisdom of the ages. She's an esteemed Terrastrid Whisperer from Aetheria, communicating with the giant beasts and understanding their needs. Lyra bears the weight of Aetheria's past, her family was torn apart by the great schism. She yearns to prevent such strife from ever happening again, seeking to bridge the gap between those who would exploit the Terrastrids and those who

## Save World

In [12]:
def save_world(world, filename):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f) 

save_world(world, f'../shared_data/{world["name"]}.json')

## Generating Text box

In [ ]:
demo = None
def start_game():
    global demo
    if demo:
        demo.close()

    with gr.Blocks() as app:
        chatbot = gr.Chatbot(
            height=300,
            type="messages",
            label="Aventura Conversacional"
        )
        msg = gr.Textbox(label="Tu acción")
        clear = gr.ClearButton([msg, chatbot])

        msg.submit(
            fn=main_loop,
            inputs=[msg, chatbot],
            outputs=[chatbot],
            queue=True
        )

    demo = app
    demo.launch(
        server_name="0.0.0.0",
        server_port=0,
        max_threads=2,
        prevent_thread_lock=True,
        share=False  # Desactivar share si hay problemas con frpc
    )
    
def test_main_loop(message, history):
    return 'Entered Action: ' + message

# start_game(test_main_loop)

## Generating Start

In [14]:
world_name = world["name"]
world = load_world(f'../shared_data/{world_name}.json')
# Access the world, kingdom, town and character. Here is the first of each selected.
#kingdom = random.choice(list(world['kingdoms'].keys()))
kingdom = list(world['kingdoms'].keys())[0] 
town = random.choice(list(world['kingdoms'][kingdom]['towns'].keys()))
character = random.choice(list(world['kingdoms'][kingdom]['towns'][town]['npcs'].keys()))

In [15]:
system_prompt = """You are an AI Game master. Your job is to create a 
start to an adventure based on the world, kingdom, town and character 
a player is playing as. 
Instructions:
You must only use 2-4 sentences \
Write in second person. For example: "You are Jack" \
Write in present tense. For example "You stand at..." \
First describe the character and their backstory. \
Then describes where they start and what they see around them."""
world_info = f"""
World: {world}
Kingdom: {kingdom}
Town: {town}
Your Character: {character}
"""

model_output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info + '\nYour Start:'}
    ],
    temperature=1.0
)
start = model_output.choices[0].message.content
print(start)
world['start'] = start
#save_world(world, f'../shared_data/{world_name}.json')  
save_world(world, f'../shared_data/{world_name}.json')

You are Orion Stargazer, an elven scholar with pale skin, silver hair, and eyes that hold the wisdom of the stars. You are an astronomer from Aetheria, studying the celestial bodies and their influence on the Terrastrids, contributing to the kingdom's vast knowledge. Despite your intellectual pursuits, you are plagued by the loss of your mentor during the great schism, driving your desire to uncover the secrets of the universe and honor his legacy. You stand in the heart of Ember's Hollow, the smell of smoke and molten metal filling the air. The Molten Forge roars nearby, its fiery glow casting long shadows across the volcanic landscape. The town bustles with blacksmiths and artisans, their hammers striking in rhythmic harmony, while the distant rumble of Ignis, the Terrastrid, echoes through the ground.


## Creating the Main Action Loop

In [39]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""
    
    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.complete(
        model=model,
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [40]:
game_state = {
    "world": world['description'],
    "kingdom": world["kingdoms"][kingdom]['description'],
    "town": world["kingdoms"][kingdom]["towns"][town]['description'],
    "character": world['kingdoms'][kingdom]['towns'][town]['npcs'][character]['description'],
    "start": start,
}

def main_loop(message, history):
    return run_action(message, history, game_state)

In [70]:
def get_game_state(game_state=game_state, *args, **kwargs):
    world = load_world(f'../shared_data/{world_name}.json')
    kingdom = game_state["kingdom"]
    town = game_state["town"]
    character = game_state["character"]
    start = game_state['start']
    inventory = {}

    game_state = {
        "world": world,
        "kingdom": kingdom,
        "town": town,
        "character": character,
        "start": start,
        "inventory": game_state.get('inventory', {})
    }
    return game_state

## Launch and Share!

In [19]:
# start_game(main_loop, True)

In [20]:
# from helper import get_game_state

# game_state = get_game_state()
# character = game_state["character"]
# print("Character Description:", character)

## Define Inventory Detector

In [42]:
system_prompt = """You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained. 
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>, 
        "change_amount": <CHANGE AMOUNT>}...
    ]
}
"""

In [62]:
def detect_inventory_changes(game_state, output):
    inventory = game_state.get('inventory', {})
    
    # Prompt más estricto para Mistral
    system_prompt = """You are an inventory tracker. Respond ONLY with valid JSON in this exact format:
{
    "itemUpdates": [
        {"name": "item_name", "change_amount": 1}
    ]
}"""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Current Inventory: {inventory}"},
        {"role": "user", "content": f"Action: {output}"},
        {"role": "user", "content": "List item changes in the specified JSON format."}
    ]
    
    # Llamada a Mistral
    response = client.chat.complete(
        model=model,
        temperature=0.0,
        messages=messages
    ).choices[0].message.content
    try:
        cleaned = response.strip()
        if cleaned.startswith('```json'):
            cleaned = cleaned[7:-3].strip()
        elif cleaned.startswith('```'):
            cleaned = cleaned[3:-3].strip()
        
        return json.loads(cleaned).get('itemUpdates', [])
    
    except Exception as e:
        print(f"Error parsing inventory: {str(e)}")
        return []


In [63]:
game_state = get_game_state(game_state)
game_state['inventory'] = {
    "cloth pants": 1,
    "cloth shirt": 1,
    "gold": 5
}

result = detect_inventory_changes(game_state, 
"You buy a sword from the merchant for 5 gold")

print(result)

[{'name': 'gold', 'change_amount': -5}, {'name': 'sword', 'change_amount': 1}]


In [64]:
def update_inventory(inventory, updates):
    try:
        if not updates:
            return ""
            
        changes = []
        for item in updates:
            name = item.get('name', '')
            amount = int(item.get('change_amount', 0))
            
            if name and amount != 0:
                inventory[name] = inventory.get(name, 0) + amount
                if inventory[name] <= 0:
                    del inventory[name]
                changes.append(f"{name} ({'+' if amount > 0 else ''}{amount})")
        
        return "Inventario actualizado: " + ", ".join(changes) if changes else ""
    except Exception as e:
        print(f"⚠️ Error en update_inventory: {str(e)}")
        return ""

## Story with inventory

In [65]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']
        
    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...) \
Don't let the player use items they don't have in their inventory.
"""

    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}
Inventory: {json.dumps(game_state['inventory'])}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]

    for user_msg, bot_resp in history:
        messages.append({"role": "assistant", "content": bot_resp})
        messages.append({"role": "user", "content": user_msg})
           
    messages.append({"role": "user", "content": message})
    if not message or not history:
        return "Estado del juego no válido"
    model_output = client.chat.complete(
        model=model,
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [66]:
game_state = get_game_state(inventory={
    "cloth pants": 1,
    "cloth shirt": 1,
    "goggles": 1,
    "leather bound journal": 1,
    "gold": 5
})

In [67]:
def main_loop(message, history):
    try:
        # Validación básica del input
        if not message.strip():
            return "Por favor escribe una acción válida"
        
        # 1. Ejecutar acción principal
        output = str(run_action(message, history, game_state))  # Aseguramos string
        
        # 2. Validar seguridad
        #if not is_safe(output):
        #    return "⚠️ Esa acción no está permitida en este momento"
        
        # 3. Procesar inventario (con reintentos)
        max_retries = 3
        inventory_msg = ""
        
        for attempt in range(max_retries):
            try:
                item_updates = detect_inventory_changes(game_state, output)
                if item_updates:
                    inventory_msg = update_inventory(game_state['inventory'], item_updates)
                break
            except Exception as e:
                if attempt == max_retries - 1:
                    print(f"⚠️ Fallo al actualizar inventario: {str(e)}")
                time.sleep(1)  # Espera entre reintentos
        
        # Construir respuesta final
        response = output
        if inventory_msg:
            response += f"\n\n🔹 {inventory_msg}"
            
        return response
        
    except Exception as e:
        # Manejo detallado del error
        error_type = type(e).__name__
        
        if str(e) == "0":
            print("🔴 Error crítico: Posible fallo en la API o respuesta vacía")
            return "El sistema no pudo procesar tu acción. Intenta algo diferente."
        else:
            print(f"🔴 Error crítico ({error_type}): {str(e)}")
            return f"Error: {error_type}. Por favor intenta nuevamente."

start_game(main_loop, True)

Closing server running on port: 7861


c:\Users\USUARIO\AppData\Local\pypoetry\Cache\virtualenvs\llm-game-xAnBDEWD-py3.13\Lib\site-packages\gradio\chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): [winerror 10048] solo se permite un uso de cada dirección de socket (protocolo/dirección de red/puerto)


* Running on local URL:  http://0.0.0.0:7861
* Running on public URL: https://81c85aa8d6003f3e7d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔴 Error crítico (ValueError): too many values to unpack (expected 2)


In [69]:
def check_api_status():
    try:
        test_response = client.chat.complete(
            model=model,
            messages=[{"role": "user", "content": "Responde 'OK'"}],
            temperature=0.0
        )
        return True if test_response.choices[0].message.content else False
    except:
        return False

if not check_api_status():
    print("🔴 La API de Mistral no está respondiendo")
    # Puedes agregar un reintento o salir

check_api_status()

True